In [1]:
using PyPlot
include("saltsolver.jl")
include("passive.jl")

/Users/davidliu/.julia/v0.4/Conda/deps/usr/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


passive_solve! (generic function with 1 method)

In [44]:
eig([1 2; 2 1])

([-1.0,3.0],
2x2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107)

In [8]:
nsym = 8
ℓ = 2
Lcav = 1.0

# uniform
wantcnv = true
σ = 0.1
ɛrpiece = [1.0]
ɛipiece = [σ]
Fpiece = [1.0]

ɛr = cn_profile(ɛrpiece, nsym)
ɛi = cn_profile(ɛipiece, nsym)
ɛ = ɛr + im*ɛi
F = cn_profile(Fpiece, nsym)

ismirror(v) = (v == ring_flip(v))
iscnv = (ismirror(ɛ) && ismirror(F))
assert(wantcnv == iscnv)

N = length(F)
assert(N == nsym)
h = Lcav / N
laplacian!(J) = periodic!(J, h)
Eguess, ωguess = ring_eigenpair(N, ℓ, h)

# purposely make it far from gain center to 
# see low-Q effects
ωa = round(ωguess) + 1
γ⟂ = 0.7
γpar = 0.1
las = Laser(ɛ, F, ωa, γ⟂);

In [3]:
# use either this panel or the one above; not both
nsym = 7
ℓ = 2
Lcav = 1.0

wantcnv = true
if wantcnv
    ɛrpiece = [1.0, 1.1, 1.1]
    ɛipiece = [0.3, 0.25, 0.25]
    Fpiece = [1.1, 0.9, 0.9]
else 
    ɛrpiece = [1.0, 1.2, 1.1]
    ɛipiece = [0.3, 0.25, 0.32]
    Fpiece = [1.1, 0.9, 0.95]
end
    
ɛr = cn_profile(ɛrpiece, nsym)
ɛi = cn_profile(ɛipiece, nsym)
ɛ = ɛr + im*ɛi
F = cn_profile(Fpiece, nsym)

ismirror(v) = (v == ring_flip(v))
iscnv = (ismirror(ɛ) && ismirror(F))
assert(wantcnv == iscnv)

N = length(F)
h = Lcav / N
laplacian!(J) = periodic!(J, h)
Eguess, ωguess = ring_eigenpair(N, ℓ, h)

# purposely make it far from gain center to 
# see low-Q effects
ωa = round(ωguess) + 1
γ⟂ = 0.7
γpar = 0.1
las = Laser(ɛ, F, ωa, γ⟂);

In [9]:
Dguess = 1.2 * mean(ɛipiece)
md = Mode(copy(Eguess), ωguess, 0.0)
Dt = threshold!(laplacian!, md, las, (Dguess, Dguess*1.05))

println("threshold search slightly breaks pure chirality")
println("use projection operator to get it back")
md = Mode(project_chiral(md.E, nsym, ℓ), md.ω, md.c²)
println("chiral-projected mode should automatically be solution")
solve!(laplacian!, md, las, Dt, isprint=true)
println("ℓ eigenvalue and residual:\n", get_ell(md.E, nsym))

fmd = mode_flip(md)
if iscnv
    println("Cnv, so flipped solved immediate")    
else
    println("Cn only, so flipped solve not immediate")
end
solve!(laplacian!, fmd, las, Dt, isprint=true)
if !iscnv
    fmd = Mode(project_chiral(fmd.E, nsym, -ℓ), fmd.ω, fmd.c²)
    println("reprojecting fmd to fix slightly broken chiral symmetry")
    println("reprojected fmd should immediately solve Newton")
    solve!(laplacian!, fmd, las, Dt, isprint=true)
end
println("ℓ eigenvalue and residual:\n", get_ell(fmd.E, nsym))

println("\ncheck that degenerate and at threshold")
println("ω₊ - ω₋ = ", md.ω - fmd.ω)
println("|c₊²|+|c₋²| = ", abs(md.c²) + abs(fmd.c²))
Es = (md.E, fmd.E)
ωt = md.ω
H, I, J, K, GD = overlap_integrals(Es, ωt, Dt, las, nsym, ℓ, Lcav);

threshold search slightly breaks pure chirality
use projection operator to get it back
chiral-projected mode should automatically be solution
|f| = 5.290873456620579e-13
ℓ eigenvalue and residual:
(2.0,1.2412670766475696e-16)
Cnv, so flipped solved immediate
|f| = 5.271275446459064e-13
ℓ eigenvalue and residual:
(-2.0,1.2412670766331194e-16)

check that degenerate and at threshold
ω₊ - ω₋ = 0.0
|c₊²|+|c₋²| = 7.208097639520836e-11


In [10]:
# CIRCULATING MODES
ω1s = [0.0, 0.0]
a²ths = [0.0, 0.0]
for m in (1, 2)
    ω1s[m] = -imag(GD / I[m]) / imag(H / I[m])
    a²ths[m] = real((ω1s[m]*H+GD) / I[m])
end

m = 2
d = 0.0001
Dlasing = Dt*(1+d)
a² = a²ths[m]
ω1 = ω1s[m]
md = Mode(copy(Es[m]), ωt+ω1*d, d * a²)
solve!(laplacian!, md, las, Dlasing, isprint=true)
Esalt = √md.c² * md.E
ωsalt = md.ω

println("expected a² = ", a²)
println("actual   a² = ", md.c² / d)
println("expected ω1 = ", ω1)
println("actual   ω1 = ", (md.ω-ωt)/d)

println("\n theoretical stability eigenvalues:")
imjh = imag(J[m]/H)
term = (abs(K[m])^2 - abs(J[m])^2) / abs(H)^2
other_val(z) = a²*(imjh + z*√(0.0im + imjh^2 + term))
println(0.0)
println(other_val(1))
println(other_val(-1))
println(2a² * imag(I[m] / H))

println("\nnumerical stability eigenvalues")
Λsmallest = smallest_stability_eigs(laplacian!, Esalt, ωsalt, Dlasing, γpar, las.ɛ)
for λ in Λsmallest
    println(λ / d)
end

|f| = 1.5617508689844456e-9
expected a² = 1.0
actual   a² = 1.0
expected ω1 = -0.0
actual   ω1 = 0.0

 theoretical stability eigenvalues:
0.0
0.0 + 0.0im
-0.838561714205012 - 0.0im
-0.838561714205012

numerical stability eigenvalues
1.5129156449291135e-7 + 0.0im
1.5140652111576358e-7 + 0.0im
-0.8392642818526799 + 0.0im
-0.8392642819890024 + 0.0im


In [41]:
# get standing mode allowed phases
θroots, ω1roots, a²standing = standing_roots_test(H, I, J, K, GD, iscnv, nsym, ℓ, maxits=15)

Cnv but not n=4|ℓ|; zeros everywhere
|K₊|+|K₋| = 4.912353430698512e-17
if K=0, then T independent of z
and ω1₊ = ω1₋ satisfied for all z because
Cnv -> I₊ = I₋)
outputting random phase angle
standing lasing mode E = |a₊|E₊ exp(iθ)|a₋|E₋
θ1 = 0.2803373042272903 × π/2
ω1[1] = 1.0764158754528914e-5
  |a₊|²[1] = 0.33875738716939496
  |a₋|²[1] = 0.3387573871693948
|a₊|²=|a₋|² because mirror symmetry


([0.440352807743811],[1.0764158754528914e-5],[(0.33875738716939496,0.3387573871693948)])

In [42]:
# for N = 4ℓ uniform z = ±i case, which is
# very special in that |a₊|²+|a₋|²=1 is the only constraint
# notice you can select a₊² anything and SALT is still solved immediately
d = 0.0001
z = 1im  # either im or -im is fine
Dlasing = Dt * (1+d)
a₊² = 0.3
assert(0 < a₊² < 1)
a₋² = 1 - a₊²
Esalt_guess = √a₊² * Es[1] + z * √a₋² * Es[2]
ωsalt_guess = ωt # uniform case ω1 = 0
md = Mode(copy(Esalt_guess), ωsalt_guess, 0.0)
c²guess = d * norm(Esalt_guess)^2 / norm(md.E)^2
md.c² = c²guess
println("there shouldn't be too many |f| here:")
solve!(laplacian!, md, las, Dlasing, isprint=true)

# hack; standing_roots function does not work for 
# this special case because T± singular
θroots = [z == 1im ? π/2 : -π/2]
ω1roots = [0]
a²standing = ((a₊², a₋²),)

# stability eigenvalues using new method specifically
# for N = 4ℓ uniform
println("theoretically predicted stability eigenvalues:")
ki = imag(K[1]/H)
2*[0, 0, ki, ki]

there shouldn't be too many |f| here:
|f| = 1.5617529687870675e-9
theoretically predicted stability eigenvalues:


4-element Array{Float64,1}:
  0.0     
  0.0     
 -0.838562
 -0.838562

In [35]:
mroot = 1
d = 0.0001
Dlasing = Dt * (1+d)
a²s = a²standing[mroot]
θroot = θroots[mroot]
Esalt_guess = √a²s[1] * Es[1] + exp(im*θroot) * √a²s[2] * Es[2]
ωsalt_guess = ωt + d * ω1roots[mroot]
md = Mode(copy(Esalt_guess), ωsalt_guess, 0.0)
c²guess = d * norm(Esalt_guess)^2 / norm(md.E)^2
md.c² = c²guess
solve!(laplacian!, md, las, Dlasing, isprint=true)
coefs = √md.c² * collect(coefficients( md.E, Es))
Esalt = √md.c² * md.E
ωsalt = md.ω
println("expected c² = ", c²guess)
println("actual   c² = ", md.c²)

println("expected ω1 = ", ω1roots[mroot])
println("actual   ω1 = ", (md.ω - ωt)/d)
for m in (1, 2)
    println("coefficient |a_", m, "|^2:")
    println("\texpected: ", a²s[m])
    println("\tactual:   ", abs(coefs[m])^2/d)
end
println("expected θ = ", θroot)
println("actual   θ = ", angle(coefs[2]/coefs[1]))

println("\nnumerical stability eigenvalues")
Λsmallest = smallest_stability_eigs(laplacian!, Esalt, ωsalt, Dlasing, γpar, las.ɛ)
for λ in Λsmallest
    println(λ / d)
end
println("if the zero eigenvalues are small but positive, try halving d and")
println("the zero eigenvalues will double, meaning it's some sort of")
println("small residual at zeroth order that is nothing to worry about")

println("\ntheoretical stability eigenvalues")
z = exp(im*θroot)
a₊a₋ = sqrt(a²s[1]*a²s[2])
a₊², a₋² = a²s 
# be careful to either say 1 + 1 or 1+1 here, not 1 +1 or else
# it will get interpreted as the next column!
XQmP̃ = [2a₋²*K[2]*z^2   a₊a₋*((z-1/z)*(I[2]+J[2]) - 2z*K[2]) ;
        -2a₊a₋/z*K[1]       a₋²*(z^2-1)*I[2] + a₊²*K[1]*(1+z^-2)     ]
XQpP̃ = [2a₊²*I[1]      a₊a₋*((z+1/z)*(I[2]+J[2]) + 2z*K[2]); 
    2a₊a₋*(z*(I[1] + J[1]) + 1/z*K[1])    a₋²*(z^2+1)*I[2] + a₊²*K[1]*(1-z^-2)] 
MatFinal = [ imag(XQpP̃/H)   -real(XQmP̃/H);
            -real(XQpP̃/H)  -imag(XQmP̃/H) ]

Λfinal, ~ = eig(MatFinal)
Λfinal = Λfinal[sortperm(abs(Λfinal))]
for λ in Λfinal
    println(λ)
end

i₊, i₋ = I[1]/H, I[2]/H
j₊, j₋ = J[1]/H, J[2]/H
k₊, k₋ = K[1]/H, K[2]/H
zr, zi = real(z), imag(z)
MatFinalAlternate_first2cols = 
2*[ a₊² * imag(i₊)      a₊a₋*(zr*imag(i₋ + j₋) + imag(z*k₋));
    a₊a₋*imag(z*(i₊ + j₊)+1/z*k₊)     a₋²*zr*imag(z*i₋) + a₊²*real(1/z*k₊)*zi;
    -a₊² * real(i₊)      -a₊a₋*(zr*real(i₋+j₋) + real(z*k₋));
    -a₊a₋*real(z*(i₊+j₊)+1/z*k₊)    -a₋²*zr*real(z*i₋) + a₊²*imag(1/z*k₊)*zi]
MatFinalAlternate_second2cols = 
2*[ -a₋²*real(k₋*z^2)   a₊a₋*(zi*imag(i₋+j₋) + real(z*k₋));
    a₊a₋*real(1/z*k₊)   a₋²*zi*imag(z*i₋) - a₊²*real(1/z*k₊)*zr;
    -a₋²*imag(k₋*z^2)   -a₊a₋*(zi*real(i₋+j₋) - imag(z*k₋));
    a₊a₋*imag(1/z*k₊)   -a₋²*zi*real(z*i₋) - a₊²*imag(1/z*k₊)*zr]
MatFinalAlternate = [MatFinalAlternate_first2cols  MatFinalAlternate_second2cols]
println("\nnorm(MatFinal - MatFinalAlternate) = ", norm(MatFinalAlternate - MatFinal))


if nsym==4abs(ℓ) && iscnv
    println("\nclosed-form theoretical predictions for Cnv with n==4ℓ:")
    println("note if uniform profiles, these will not be correct")
    isrealz = iseven(round(Int, θroot/(π/2)))
    println("standing solution with z = ", isrealz ? "±1" : "±i")
    vals = [0.0, 0.0, 0.0, 0.0] + 0im
    vals[1] = 0.0
    
    α = isrealz ? 1 : -1
    # I have no idea what to name this!
    
    vals[2] = 2a₊² * imag(2i₊ + j₊ + α*k₊)
    radicand = 0.0im + imag(k₊ - α * j₊)^2 - α * 8real(k₊)*real(j₊ + α * k₊)
    vals[3] = -a₊² * imag(j₊ + α * 3k₊) + a₊² * √radicand
    vals[4] = -a₊² * imag(j₊ + α * 3k₊) - a₊² * √radicand
    abslt(x, y) = abs(x) < abs(y)
    sort!(vals, lt=abslt)
    for λ in vals
        println(λ)
    end
    
    for sn in (1, -1)
        Mat = nothing
        if isrealz
            Mat = 2a₊² * [imag(i₊)   sn*imag(i₋+j₋+k₋)   -√2*real(k₋);
                sn*imag(i₊+j₊+k₊)    imag(i₋)         √2*sn*real(k₊);
                1/√2*real(j₊+k₊)       -1/√2*sn*real(j₋+k₋)  -imag(k₋)-imag(k₊)]
        else
            Mat = 2a₊² * [
                imag(i₊)    sn*√2*real(k₋)    sn*imag(i₋+j₋-k₋);
                sn/√2*real(j₊-k₊)   imag(k₊+k₋)   -1/√2*real(j₋-k₋);
                sn*imag(i₊+j₊-k₋)   -√2*real(k₊)   imag(i₋)
            ]
        end
        vals, ~ = eig(Mat)
        sort!(vals, lt=abslt)
        println("theoretical eigenvalues from 3x3 problem for z = ", sn*(isrealz ? 1 : 1im))
        for λ in vals
            println("\t", λ)
        end
    end
elseif iscnv && nsym!=4abs(ℓ)
    println("\nclosed-form theoretical predictions for Cnv with n!=4ℓ:")
    println("two nonzero eigenvalues")
    vals = [0.0, 0.0]
    ss = [1, -1]
    tr0 = 2imag(a₊² * i₊ + a₋² * i₋)
    det0 = -4a₊a₋^2*(imag(i₊)*imag(j₋) + imag(j₊)*imag(i₋) + imag(j₊)*imag(j₋))
    for m = (1, 2)
        vals[m] = tr0/2 + ss[m] * sqrt((tr0/2)^2 - det0)
    end
    # 2a₊²*(imag(i₊) + 2abs(imag(i₊)))
    # also works for TM (J=I), and is provably both signs
    for λ in vals
        println(λ)
    end
end

|f| = 1.5617516205102857e-9
expected c² = 0.0001
actual   c² = 0.0001
expected ω1 = 0
actual   ω1 = 0.0
coefficient |a_1|^2:
	expected: 0.273
	actual:   0.27299999999999996
coefficient |a_2|^2:
	expected: 0.727
	actual:   0.7269999999999999
expected θ = 1.5707963267948966
actual   θ = 1.5707963267948968

numerical stability eigenvalues
1.5133863214992654e-7 + 0.0im
1.5139523894025716e-7 + 0.0im
-0.8392642819237492 + 0.0im
-0.8392642820263274 + 0.0im
if the zero eigenvalues are small but positive, try halving d and
the zero eigenvalues will double, meaning it's some sort of
small residual at zeroth order that is nothing to worry about

theoretical stability eigenvalues
-5.76985678950921e-17 + 0.0im
1.005787638168938e-16 + 0.0im
-0.838561714205012 + 2.5257797458649368e-17im
-0.838561714205012 - 2.5257797458649368e-17im

norm(MatFinal - MatFinalAlternate) = 1.308944633364907e-16

closed-form theoretical predictions for Cnv with n==4ℓ:
note if uniform profiles, these will not be correct
st